# Data Smart Sans Excel

[Table of Contents](Data%20Smart%20Sans%20Excel.ipynb)

If you have not downloaded the Data Smart files then run the first code block of the main notebook and it will download the files from the web. 

## Chapter 4 - Optimization Modeling

In [1]:
import os
import pandas as pd
import numpy as np
excel_file = os.path.join(os.getcwd(), "data_smart_files", "ch04", "OrangeJuiceBlending.xlsx")
specs_df = pd.read_excel(excel_file, 'Specs', parse_cols = "A:I", index_col=0)

* Objective: Minimize procurement costs.
* Decisions: Amount of each juice to buy each month
* Constraints:
    * Demand (600k in Jan/Feb, 700k in Mar)
    * Supply
    * Florida Valencia requirement: Buy 40% of juice from FL.
    * Flavor: 
        * 11.5 < BAR < 12.5
        * 0.75 < Acid < 1
        * Astrigency <= 4
    * Color: 4.5 < Color < 5.5

In [2]:
print (specs_df)

                    Region  Qty Available (1,000 Gallons)  Brix / Acid Ratio  \
Varietal                                                                       
Hamlin              Brazil                            672               10.5   
Mosambi              India                            400                6.5   
Valencia           Florida                           1200               12.0   
Hamlin          California                            168               11.0   
Gardner            Arizona                             84               12.0   
Sunstar              Texas                            210               10.0   
Jincheng             China                            588                9.0   
Berna                Spain                            168               15.0   
Verna               Mexico                            300                8.0   
Biondo Commune       Egypt                            210               13.0   
Belladonna           Italy              

In [3]:
total_required = [600, 600, 700]
valencia_required = [0.4*x for x in total_required]
    
quality_cons_min = [11.5, 0.0075, 0, 4.5]
quality_cons_max = [12.5, 0.01, 4, 5.5]

In [21]:
def opti(x, baseline_data):
    opti_model = baseline_data.copy()
    
    if opti_model.shape[0] == 11: # 11 rows 
        opti_model['Jan Orders'] = x[:11]
        opti_model['Feb Orders'] = x[11:22]
        opti_model['Mar Orders'] = x[22:33] 

    opti_model['Jan Price'] = opti_model['Jan Orders'] * opti_model['Price (per 1K Gallons)']
    opti_model['Feb Price'] = opti_model['Feb Orders'] * opti_model['Price (per 1K Gallons)']
    opti_model['Mar Price'] = opti_model['Mar Orders'] * opti_model['Price (per 1K Gallons)']
    
    opti_model['Jan Shipping'] = opti_model['Jan Orders'] * opti_model['Shipping']
    opti_model['Feb Shipping'] = opti_model['Feb Orders'] * opti_model['Shipping']
    opti_model['Mar Shipping'] = opti_model['Mar Orders'] * opti_model['Shipping']
   
    opti_model.loc['Total'] = opti_model.sum(numeric_only=True)
    
    total_cost = opti_model.at['Total','Jan Price'] + opti_model.at['Total','Feb Price'] + opti_model.at['Total','Mar Price'] + opti_model.at['Total', 'Jan Shipping'] + opti_model.at['Total','Feb Shipping'] + opti_model.at['Total','Mar Shipping']
    
    return total_cost

In [5]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec

def plot_init():
    fig = plt.figure(figsize=(10, 4))
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 2]) 
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1])
    ax0.set_xlabel('Time')
    ax0.set_ylabel('Total Distance')
    #ax1.set_xlabel('Offers')
    #ax1.set_ylabel('Clusters')
    #ax1.set_xlim(-0.5, 0.5)
    #ax1.set_ylim(-0.5,0.5)
    return fig, [ax0, ax1]

In [24]:
from scipy.optimize import minimize

def orders_check(o, req):
    return o.sum() - req

def qual_cons_compare(orders,baseline, measure, mvm, constraint, total):
    return mvm*(constraint - ((orders * baseline[measure]).sum() / total))

#fig, [ax0, ax1] = plot_init()  

cons = [{'type': 'ineq', 'fun': lambda x: x}, # order is positive
        {'type': 'eq', 'fun': lambda x: orders_check(x[:11],total_required[0])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[11:22],total_required[1])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[22:33],total_required[2])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[2],valencia_required[0])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[13],valencia_required[1])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[24],valencia_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Brix / Acid Ratio',1,quality_cons_max[0],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Acid (%)',1,quality_cons_max[1],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Astringency (1-10 Scale)',1,quality_cons_max[2],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Color (1-10 Scale)',1,quality_cons_max[3],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Brix / Acid Ratio',1,quality_cons_max[0],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Acid (%)',1,quality_cons_max[1],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Astringency (1-10 Scale)',1,quality_cons_max[2],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Color (1-10 Scale)',1,quality_cons_max[3],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Brix / Acid Ratio',1,quality_cons_max[0],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Acid (%)',1,quality_cons_max[1],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Astringency (1-10 Scale)',1,quality_cons_max[2],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Color (1-10 Scale)',1,quality_cons_max[3],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Brix / Acid Ratio',-1,quality_cons_min[0],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Acid (%)',-1,quality_cons_min[1],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Astringency (1-10 Scale)',-1,quality_cons_min[2],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Color (1-10 Scale)',-1,quality_cons_min[3],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Brix / Acid Ratio',-1,quality_cons_min[0],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Acid (%)',-1,quality_cons_min[1],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Astringency (1-10 Scale)',-1,quality_cons_min[2],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Color (1-10 Scale)',-1,quality_cons_min[3],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Brix / Acid Ratio',-1,quality_cons_min[0],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Acid (%)',-1,quality_cons_min[1],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Astringency (1-10 Scale)',-1,quality_cons_min[2],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Color (1-10 Scale)',-1,quality_cons_min[3],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: specs_df['Qty Available (1,000 Gallons)'].values - x[:11]},
        {'type': 'ineq', 'fun': lambda x: specs_df['Qty Available (1,000 Gallons)'].values - x[11:22]},
        {'type': 'ineq', 'fun': lambda x: specs_df['Qty Available (1,000 Gallons)'].values - x[22:33]}]

x0 = np.zeros(33)
x0[:11] = total_required[0] / 11
x0[11:22] = total_required[1] / 11
x0[22:33] = total_required[2] / 11

res = minimize(opti, x0, args=(specs_df), constraints=cons)

print (res)

orders = res.x
results = specs_df.copy()
results['Jan Orders'] = orders[:11]
results['Feb Orders'] = orders[11:22]
results['Mar Orders'] = orders[22:33] 
results['Jan Price'] = results['Jan Orders'] * results['Price (per 1K Gallons)']
results['Feb Price'] = results['Feb Orders'] * results['Price (per 1K Gallons)']
results['Mar Price'] = results['Mar Orders'] * results['Price (per 1K Gallons)']
results['Jan Shipping'] = results['Jan Orders'] * results['Shipping']
results['Feb Shipping'] = results['Feb Orders'] * results['Shipping']
results['Mar Shipping'] = results['Mar Orders'] * results['Shipping']
results.loc['Total'] = results.sum(numeric_only=True)
results.loc['Brix / Acid Ratio'] = 0
results.loc['Acid (%)'] = 0
results.loc['Astringency (1-10 Scale)'] = 0
results.loc['Color (1-10 Scale)'] = 0

for month in ['Jan', 'Feb', 'Mar']:
    month_orders = month + ' Orders'
    for constraint in ['Brix / Acid Ratio', 'Acid (%)', 'Astringency (1-10 Scale)', 'Color (1-10 Scale)']:
        results.at[constraint,month_orders] = (results[month_orders] * results[constraint]).sum() / results.at['Total',month_orders]

results.drop(['Region', 'Qty Available (1,000 Gallons)', 'Brix / Acid Ratio',
              'Acid (%)', 'Astringency (1-10 Scale)', 'Color (1-10 Scale)',
              'Price (per 1K Gallons)', 'Shipping', 'Jan Price', 'Feb Price', 
              'Mar Price', 'Jan Shipping', 'Feb Shipping', 'Mar Shipping'], axis=1, inplace=True)
    
print (results)

     fun: 1177124.9999895953
     jac: array([ 600.,  460.,  750.,  660.,  675.,  675.,  560.,  710.,  390.,
        590.,  620.,  600.,  460.,  750.,  660.,  675.,  675.,  560.,
        710.,  390.,  590.,  620.,  600.,  460.,  750.,  660.,  675.,
        675.,  560.,  710.,  390.,  590.,  620.,    0.])
 message: 'Optimization terminated successfully.'
    nfev: 281
     nit: 9
    njev: 8
  status: 0
 success: True
       x: array([ -2.19418253e-10,   5.03256742e-09,   2.40000000e+02,
        -2.47534799e-09,   8.13356256e+00,   7.65591111e+00,
         1.25711589e-09,  -4.35370443e-09,   1.26315789e+02,
         1.09736842e+02,   1.08157895e+02,  -2.25156704e-10,
         5.03117084e-09,   2.40000000e+02,  -2.47760066e-09,
         8.13356256e+00,   7.65591111e+00,   1.27662570e-09,
        -4.35457783e-09,   1.26315789e+02,   1.09736842e+02,
         1.08157895e+02,  -2.25386718e-10,   5.03116698e-09,
         2.80000000e+02,  -2.47748364e-09,   9.21052631e+00,
         9.21052631e

In [ ]:
#TODO: I am getting an error. Simply put, the function isnt solvable. It is likely an issue in the constraints. 
#As it is written it is hard to debug. Next step is to add plotting of some of the variables to show them evolve.
#It seems that the best way to do this is to make a class allowing me to share a variable with opti each time it runs. 

#I made some tweaks (fixed a few constraints) and it runs successfully (success: True), but I am getting negative orders
#even though I specifically made a constraint for orders to be positive. 